# Tarea 3: Evaluación de Segmentación


## 1. Importaciones y Configuración

In [ ]:
# 1. Importaciones y Configuración
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage import segmentation
print("Todas las dependencias están correctamente instaladas")

## 2. Funciones de Carga y Visualización

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import graph, segmentation

# Load the ground truth image
image = cv2.imread("../dataset/malignant/malignant (2).png", cv2.IMREAD_GRAYSCALE)

# Normalize the image (enhances contrast for better segmentation)
image = cv2.equalizeHist(image)

# Display the image
plt.imshow(image, cmap='gray')
plt.title("Ground Truth Image")
plt.show()




Define the Split and Merge Algorithm

In [3]:
def split(image, x, y, width, height, threshold):
    """ Recursively split the image into quadrants """
    if width < 2 or height < 2:  
        return [(x, y, width, height)]
    
    # Extract region
    region = image[y:y+height, x:x+width]
    
    # Compute homogeneity criterion (variance)
    mean = np.mean(region)
    std = np.std(region)
    
    if std < threshold:  
        return [(x, y, width, height)]
    
    # Split into four quadrants
    half_w, half_h = width // 2, height // 2
    regions = []
    regions += split(image, x, y, half_w, half_h, threshold)
    regions += split(image, x + half_w, y, half_w, half_h, threshold)
    regions += split(image, x, y + half_h, half_w, half_h, threshold)
    regions += split(image, x + half_w, y + half_h, half_w, half_h, threshold)

    return regions


Merge Similar Regions

In [4]:
def merge(image, regions, threshold):
    """ Merge similar regions based on mean intensity """
    merged_regions = []
    visited = set()

    for i, (x1, y1, w1, h1) in enumerate(regions):
        if i in visited:
            continue

        mean1 = np.mean(image[y1:y1+h1, x1:x1+w1])
        merged = False

        for j, (x2, y2, w2, h2) in enumerate(regions):
            if i != j and j not in visited:
                mean2 = np.mean(image[y2:y2+h2, x2:x2+w2])
                
                if abs(mean1 - mean2) < threshold:
                    # Merge regions
                    min_x, min_y = min(x1, x2), min(y1, y2)
                    max_x, max_y = max(x1+w1, x2+w2), max(y1+h1, y2+h2)
                    merged_regions.append((min_x, min_y, max_x-min_x, max_y-min_y))
                    visited.add(i)
                    visited.add(j)
                    merged = True
                    break

        if not merged:
            merged_regions.append((x1, y1, w1, h1))

    return merged_regions


Apply Split and Merge on the Image

In [ ]:
# Set homogeneity threshold
threshold_split = 15
threshold_merge = 10

# Perform splitting
regions = split(image, 0, 0, image.shape[1], image.shape[0], threshold_split)

# Perform merging
merged_regions = merge(image, regions, threshold_merge)

# Visualizing result
output = np.zeros_like(image)
for x, y, w, h in merged_regions:
    output[y:y+h, x:x+w] = np.mean(image[y:y+h, x:x+w])  # Assign mean intensity

plt.imshow(output, cmap='gray')
plt.title("Segmented Image after Split and Merge")
plt.show()
